In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp "/content/drive/My Drive/Datasets/kaggle_rock_endings.csv" "kaggle_rock_endings.csv"
!cp "/content/drive/My Drive/Datasets/kaggle_rock_endings_generated.csv" "kaggle_rock_endings_generated.csv"

Mounted at /content/drive


In [ ]:
import pandas as pd
realEndings = pd.read_csv("kaggle_rock_endings.csv")
generatedEndings = pd.read_csv("kaggle_rock_endings_generated.csv")
generatedEndings.head()

,Unnamed: 0,endings
0,0,i know there is a place in your heart\nfor you...
1,1,night like this\na night like this\ni was driv...
2,2,a ride tonight\nand make it out of this town\n...
3,3,on a cloud\ncant nobody tell me what to do\ni ...
4,4,night\nif you can't understand\nwell you can't...


In [ ]:
endings = realEndings["endings"].values.tolist()
generatedEndings = generatedEndings["endings"].values.tolist()
generatedEndings

["i know there is a place in your heart\nfor you to take me back again\nyou're the one who made my world come down\ni wanna be around\nnow that's what i'm here for you\ni'd do anything to make you smile\nit's all you and me oh baby\neven though i just can't get enough of you\nmy heart still beats in time with you\nno more holding on to you\nin this world where no one cares\nwherever you may be\nwe'll always be together\ni'll never let you go\ni'll keep holding on to you\nif i never ever ever ever get to see you again\nremember it was you that made me smile\nyou satisfy the need you created\nnot for the first time could be for real",
 "night like this\na night like this\ni was driving into town the first day of july\nthe very first day i met the girl of my dreams\nwe had to make a stand for something that was not so good\nyou walked up and said this is not my town\nwhy'd we even try?\nyou were so beautiful\nbut it was everything\nthat you'd ever want\nit's not your fault but its not you

In [ ]:
endingsWordList = [str(ending).split() for ending in endings[:20]]

In [ ]:
generatedEndingsWordList = [str(genEnding).split() for genEnding in generatedEndings]

In [ ]:
print(endings[0])
print()
print(generatedEndings[0])

in times of desperation when i needed a friend
remember it was you that made it all worthwhile
i sometimes try to fake hesitation
not for the first time i've started to feel
straight straight through my heart those arrows fly
straight straight through my heart and that's no lie
i don't want to play the innocent you know me too well
you still keep me wondering but i don't know why
you satisfy the need you created
and for the first time i feel so real
straight straight through my heart those arrows fly
straight straight through my heart and that's no lie
wait wait for me please don't pass me by
if this world's without you i'd surely die
i'm in the middle of a love without an end
reality too easy to comprehend
straight straight through my heart those arrows fly
straight straight through my heart and that's no lie
wait wait for me please don't pass me by
if this world's without you i'd surely die
i'm in the middle of a love without an end
reality too easy to comprehend
kono yono nani yori 

BLEU SCORE:

In [ ]:
# the bleu score has its origins in translation however it can be applied also to text generation
# especially the 4-gram score is important for us as we do not want word to word translation 
# but rather for gpt2 to capture the theme and style and then generete similar but not identical songs
from nltk.translate.bleu_score import corpus_bleu
references = endingsWordList
candidates = generatedEndingsWordList

In [ ]:
print('Cumulative 1-gram: %f' % corpus_bleu(references, candidates, weights=(1, 0, 0, 0)))
print('Cumulative 2-gram: %f' % corpus_bleu(references, candidates, weights=(0.5, 0.5, 0, 0)))
print('Cumulative 3-gram: %f' % corpus_bleu(references, candidates, weights=(0.33, 0.33, 0.33, 0)))
print('Cumulative 4-gram: %f' % corpus_bleu(references, candidates, weights=(0.25, 0.25, 0.25, 0.25)))

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


Cumulative 1-gram: 0.019108
Cumulative 2-gram: 0.138233
Cumulative 3-gram: 0.270896
Cumulative 4-gram: 0.371797


COSINE SIMILARITY with GPT Embeddings

In [ ]:
#as embeddings are trained to be similar in case of similar words it makes sense to try to compute the cosine similarity of the embeddings of the texts
from scipy import spatial
!pip install transformers
from transformers import GPT2Tokenizer

     |████████████████████████████████| 2.6 MB 5.5 MB/s 
     |████████████████████████████████| 3.3 MB 36.4 MB/s 
     |████████████████████████████████| 636 kB 41.5 MB/s 
     |████████████████████████████████| 895 kB 51.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
tokenizedEndings = [tokenizer.encode(str(ending)) for ending in endings[:20]]
tokenizedGeneratedEndings = [tokenizer.encode(str(genEnding)) for genEnding in generatedEndings]


In [ ]:
average = 0
for i, ending in enumerate(tokenizedEndings): 
    minimumLength = min(len(ending), len(tokenizedGeneratedEndings[i]))
    average += 1 - spatial.distance.cosine(tokenizedGeneratedEndings[i][:minimumLength], ending[:minimumLength])

average = average / len(tokenizedEndings)

print(f"cosine similarity with gpt2 encoder: {average}")

cosine similarity with gpt2 encoder: 0.20042461300705225


COSINE SIMILARITY with BERT Embeddings

In [ ]:
#I wanted to cross validate the results with other embeddings 
#As the model was trained on gpt2 I expected more similar results with gpt2
#this was to my surprise not the case

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
tokenizedEndings = [tokenizer.encode(str(ending)[:512]) for ending in endings[:20]]
tokenizedGeneratedEndings = [tokenizer.encode(str(genEnding)[:512]) for genEnding in generatedEndings]


In [ ]:
average = 0
for i, ending in enumerate(tokenizedEndings): 
    minimumLength = min(len(ending), len(tokenizedGeneratedEndings[i]))
    average += 1 - spatial.distance.cosine(tokenizedGeneratedEndings[i][:minimumLength], ending[:minimumLength])

average = average / len(tokenizedEndings)

print(f"cosine similarity with bert encoder: {average}")

cosine similarity with bert encoder: 0.5839559150566088


Percentage difference in text length


In [ ]:
# i noticed that gpt2 tends to generate shorter texts and I wanted to see how much shorter they actually were
percentDifference = 0
for i, ending in enumerate(endings[:20]): 
  percentDifference += abs(len(generatedEndings[i]) - len(ending[:250])) / ((len(generatedEndings[i])+len(ending[:250]))/2)

percentDifference = percentDifference *100 / len(endings[:20])
print(f"Percentage difference in text length: {percentDifference} %")

Percentage difference in text length: 74.96712981963427 %
